<a href="https://colab.research.google.com/github/iLab-DSU/Imarika_Voice_Analytics/blob/main/STT_using_CTC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Speech to Text using CTC

### Introduction 
- Making use of RNN, 2D CNN and Connectionist Temporal Classification loss to build a STT model
- CTC is used to train deep neural networks in speech recog, handwriting recog, etc.
- We will use the Common Voice dataset from [here](https://commonvoice.mozilla.org/sw/datasets).
- The quality of the model will be evaluated using Word Error Rate(WER). 



### Library Installations

In [1]:
pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install -U datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Setup Required Libraries

In [29]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers


### Data Pipeline

In [4]:
# Login huggingface


from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [5]:
# Download the dataset from huggingface

from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("mozilla-foundation/common_voice_8_0", 'sw', split='train+validation', use_auth_token=True)
common_voice_test = load_dataset("mozilla-foundation/common_voice_8_0", 'sw', split='test', use_auth_token=True)

In [6]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "down_votes", "gender", "locale", "segment", "up_votes"])

In [7]:
common_voice_train

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence'],
    num_rows: 28411
})

In [8]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML


def display_random_elements(dataset, num_examples=10):
  assert num_examples <= len(dataset) # Can't pick more than there is in the dataset
  picks = []
  for _ in range(num_examples):
      pick = random.randint(0, len(dataset)-1)
      while pick in picks:
        pick = random.randint(0, len(dataset)-1)
      picks.append(pick)

  
  df = pd.DataFrame(dataset[picks])
  display(HTML(df.to_html()))

In [9]:
import re
characters_to_ignore = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_chars(batch):
  batch['sentence']=re.sub(characters_to_ignore, '', batch['sentence']).lower() + " "
  return batch

In [10]:
common_voice_train = common_voice_train.map(remove_special_chars)
common_voice_test = common_voice_test.map(remove_special_chars)

In [11]:
display_random_elements(common_voice_train.remove_columns(['client_id']))

/usr/local/lib/python3.7/dist-packages/datasets/features/audio.py:310: UserWarning: Decoding mp3 with `librosa` instead of `torchaudio`, decoding is slow.
  warnings.warn("Decoding mp3 with `librosa` instead of `torchaudio`, decoding is slow.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/datasets/features/audio.py:310: UserWarning: Decoding mp3 with `librosa` instead of `torchaudio`, decoding is slow.
  warnings.warn("Decoding mp3 with `librosa` instead of `torchaudio`, decoding is slow.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/datasets/features/audio.py:310: UserWarning: Decoding mp3 with `librosa` instead of `torcha

,path,audio,sentence
0,/root/.cache/huggingface/datasets/downloads/extracted/53d30892a707d521ef30c6c48f4ea9f8232daea92c37971ac64fb135aa3d6acf/cv-corpus-8.0-2022-01-19/sw/clips/common_voice_sw_29840493.mp3,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/53d30892a707d521ef30c6c48f4ea9f8232daea92c37971ac64fb135aa3d6acf/cv-corpus-8.0-2022-01-19/sw/clips/common_voice_sw_29840493.mp3', 'array': [-2.1230822e-08, 1.3050334e-08, 1.8151752e-08, -3.176808e-08, 1.0266302e-09, 4.0967482e-08, -3.344724e-08, -2.7060414e-08, 6.4254635e-08, -1.5188792e-08, -7.025631e-08, 7.359241e-08, 3.2043463e-08, -1.1722196e-07, 4.985759e-08, 1.0798698e-07, -1.4449782e-07, -2.3347907e-08, 1.9598754e-07, -1.2040503e-07, -1.4873558e-07, 2.600356e-07, -1.5707004e-08, -3.0456735e-07, 2.495301e-07, 1.8083888e-07, -4.3919104e-07, 1.1285589e-07, 4.4688537e-07, -4.7487205e-07, -1.8313725e-07, 7.166971e-07, -3.2071117e-07, -6.3443616e-07, 8.798609e-07, 1.13132046e-07, -1.1901271e-06, 7.802035e-07, 9.2296324e-07, -1.7577239e-06, 1.666664e-07, 2.3301855e-06, -2.224334e-06, -1.754231e-06, 5.764305e-06, -2.4878939e-06, -2.3877134e-05, -2.3877134e-05, -2.4878939e-06, 5.764305e-06, -1.754231e-06, -2.224334e-06, 2.3301855e-06, 1.666664e-07, -1.7577239e-06, 9.2296324e-07, 7.802035e-07, -1.1901271e-06, 1.13132046e-07, 8.798609e-07, -6.3443616e-07, -3.2071117e-07, 7.166971e-07, -1.8313725e-07, -4.7487205e-07, 4.4688537e-07, 1.1285589e-07, -4.3919104e-07, 1.8083888e-07, 2.495301e-07, -3.0456735e-07, -1.5707004e-08, 2.600356e-07, -1.4873558e-07, -1.2040503e-07, 1.9598754e-07, -2.3347907e-08, -1.4449782e-07, 1.0798698e-07, 4.985759e-08, -1.1722196e-07, 3.2043463e-08, 7.359241e-08, -7.025631e-08, -1.5188792e-08, 6.4254635e-08, -2.7060414e-08, -3.344724e-08, 4.0967482e-08, 1.0266302e-09, -3.176808e-08, 1.8151752e-08, 1.3050334e-08, -2.1230822e-08, 2.865725e-09, 1.3874331e-08, -1.0184482e-08, -4.067796e-09, 9.61578e-09, -2.662255e-09, ...], 'sampling_rate': 48000}",walikuja kufanya biashara na wenyeji wa bara la afrika
1,/root/.cache/huggingface/datasets/downloads/extracted/53d30892a707d521ef30c6c48f4ea9f8232daea92c37971ac64fb135aa3d6acf/cv-corpus-8.0-2022-01-19/sw/clips/common_voice_sw_29972797.mp3,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/53d30892a707d521ef30c6c48f4ea9f8232daea92c37971ac64fb135aa3d6acf/cv-corpus-8.0-2022-01-19/sw/clips/common_voice_sw_29972797.mp3', 'array': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...], 'sampling_rate': 48000}",magonjwa yanayotibika na yasiyotibika kama kansa ukimwi na kifua kikuu
2,/root/.cache/huggingface/datasets/downloads/extracted/53d30892a707d521ef30c6c48f4ea9f8232daea92c37971ac64fb135aa3d6acf/cv-corpus-8.0-2022-01-19/sw/clips/common_voice_sw_29995022.mp3,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/53d30892a707d521ef30c6c48f4ea9f8232daea92c37971ac64fb135aa3d6acf/cv-corpus-8.0-2022-01-19/sw/clips/common_voice_sw_29995022.mp3', 'array': [-2.1230822e-08, 1.3050334e-08, 1.8151752e-08, -3.176808e-08, 1.0266302e-09, 4.0967482e-08, -3.344724e-08, -2.7060414e-08, 6.4254635e-08, -1.5188792e-08, -7.025631e-08, 7.359241e-08, 3.204123e-08, -1.1724493e-07, 4.9890982e-08, 1.080117e-07, -1.446032e-07, -2.3279101e-08, 1.961598e-07, -1.207209e-07, -1.4876133e-07, 2.6065558e-07, -1.6247341e-08, -3.051596e-07, 2.5100297e-07, 1.8050027e-07, -4.4130857e-07, 1.1527466e-07, 4.4811716e-07, -4.797041e-07, -1.8086668e-07, 7.2205455e-07, -3.286332e-07, -6.356494e-07, 8.9239023e-07, 1.04462785e-07, -1.2007823e-06, 8.0116746e-07, 9.2068166e-07, -1.7848425e-

In [15]:
common_voice_test

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence'],
    num_rows: 8941
})

### Data Preprocessing

#### Vocabulary to be used

In [18]:
# Set of characters acceptable in swahili

characters = [x for x in "abcdefghijklmnopqrstuvwxyz'?! "]

# Mapping characters to integers
char_to_num = keras.layers.StringLookup(vocabulary=characters, oov_token="")

# Mapping integers to characters
num_to_char = keras.layers.StringLookup(vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True) 


print(
    f"The vocabulary is: {char_to_num.get_vocabulary()}"
    f"(size = {char_to_num.vocabulary_size()}"
)

The vocabulary is: ['', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'", '?', '!', ' '](size = 31


In [35]:
wavs_path = "/content/wav/"
train_df_path = "/content/train.tsv"

train_df = pd.read_csv(train_df_path, sep='\t')
train_df = train_df[['path', 'sentence']]
train_df.head()

,path,sentence
0,common_voice_sw_29914941.mp3,au mabadiliko ya utandawazi yanachangia katika...
1,common_voice_sw_29914942.mp3,james aliibuka mfungaji wa bao bora la mashind...
2,common_voice_sw_29914944.mp3,Haya yanapaswa kutunzwa na kutumiwa ipasavyo k...
3,common_voice_sw_29914946.mp3,Kitaifa maadhimisho haya yalifanyika katika Ki...
4,common_voice_sw_29914947.mp3,Ingawa ina fonimu nyingi za Kikushi


In [36]:
# Window lenght in samples
frame_length = 256

# Number of samples to step
frame_step = 160

# Sise of the Fast Fourier Transform(FFT) to apply
fft_length = 384


def encode_single_sample(wav_file, label):
  ## Process the Audio

  # 1. Read wav file
  file = tf.io.read_file(wavs_path + wav_file + ".wav")

  # 2. Decode the wav file
  audio, _ = tf.audio.decode_wav(file)
  audio = tf.squeeze(audio, axis=-1)

  # 3. Change type to float
  audio = tf.cast(audio, tf.float32)

  # 4. Get the spectogram
  spectogram = tf.signal.stft(
      audio, frame_length=frame_length, frame_step=frame_step, fft_length=fft_length
  )

  # 5. Applying tf.abs to get magnitude of audios
  spectogram = tf.abs(spectogram)
  spectogram = tf.math.pow(spectogram, 0.5)

  # 6. Normalisation
  means = tf.math.reduce_mean(spectogram, 1, keepdims=True)
  stddevs = tf.math.reduce_std(spectogram, 1, keepdims=True)
  spectogram = (spectogram - means) / (stddevs + 1e-10)

  # Process the label
  
  # 7. Convert labels to lower case
  label = tf.strings.lower(label)

  # 8. Split the label
  label = tf.strings.unicode_split(label, input_encoding='UTF-8')

  # 9. Map the characters in label to numbers
  label = char_to_num(label)

  # 10. Return a dictionary since our model ecpects 2 inputs
  return spectogram, label


#### Create dataset objects


Create a tf.data.Dataset object that yield the transformed elements in the same order as they appear in the input

In [37]:
batch_size = 32

# Define training dataset

train_dataset = tf.data.Dataset.from_tensor_slices(
    (list(train_df['path']), list(train_df['sentence']))
)


train_dataset = (
    train_dataset.map(encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE)
    .padded_batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

# Define the validation dataset

validation_dataset = tf.data.Dataset.from_tensor_slices(
    (list(common_voice_test['path']), list(common_voice_test['sentence']))
)

validation_dataset = (
    validation_dataset.map(encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE)
    .padded_batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

In [ ]:
common_voice_test['path']

#### Visualise the Data

In [40]:
fig = plt.figure(figsize=(8,5))

for batch in train_dataset.take(1):
  spectogram = batch[0][0].numpy()
  spectogram = np.array([np.trim_zeros(x) for x in np.transpose(spectogram)])
  label = batch[1][0]

  # Spectogram
  label = tf.strings.reduce_join(num_to_char(label)).numpy().decode("utf-8")
  ax = plt.subplot(2, 1, 1)
  ax.imshow(spectogram, vmax=1)
  ax.set_title(label)
  ax.axis("off")

  # Wav
  file = tf.io.read_file(wavs_path + list(train_df['path'])[0] + ".wav")
  audio, _ = tf.audio.decode_wav(file)
  audio = audio.numpy()
  ax = plt.subplot(2, 1, 2)
  plt.plot(audio)
  ax.set_title("Signal Wave")
  ax.set_xlim(0, len(audio))
  display.display(display.Audio(np.transpose(audio), rate=16000))

plt.show()

NotFoundError: ignored

<Figure size 576x360 with 0 Axes>